In [ ]:
import pandas as pd
import matplotlib.colors as mc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import yaml
import pypsa
import calendar

from matplotlib.cm import ScalarMappable
from pypsa.descriptors import Dict

### Select run & Prepare data

In [ ]:
def load_configuration(config_path):
    """
    Load configuration settings from a YAML file.
    """
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config


snakemake = Dict()
snakemake.config = load_configuration("../config.yaml")
snakemake.input = Dict()
snakemake.output = Dict()

In [ ]:
run = "test-IEDK-1H-cfe100-allflex"  # run name from config.yaml

if True:
    folder = f"/results/{run}"
    scenario = "/2025/IEDK/p1/cfe100"

    snakemake.input.data = f"{folder}/networks/{scenario}/40.nc"
    snakemake.output.plot = f"{folder}/plots/plot.pdf"

    n = pypsa.Network(f"../{folder}/networks/{scenario}/40.nc")

### Explore the data

In [ ]:
n.generators_t.keys()

In [ ]:
# display two dataframes containing the time series
df = n.generators_t.p_max_pu.filter(regex="Ireland|Denmark")
df

In [ ]:
df = (
    n.generators_t.p_max_pu.filter(regex="Ireland|Denmark")
    .reset_index()
    .rename(columns={"index": "snapshot"})
    .assign(snapshot=lambda x: pd.to_datetime(x["snapshot"]))
)
df

### Visualise time-series of RES feed-in

In [ ]:
def prepare_heatmap_data(df, month, location, carrier, scaling, diff_location=None):
    data = df[df["snapshot"].dt.month == month]
    day = data["snapshot"].dt.day
    value = data[f"{location} {carrier}"].values

    if diff_location:
        diff_value = data[f"{diff_location} {carrier}"].values
        value -= diff_value  # Subtracting the second time-series from the first

    value = value.reshape(int(24 / scaling), len(day.unique()), order="F")
    return day, value


def draw_heatmap(ax, day, value, scaling, colormap, min_val, max_val):
    xgrid = np.arange(day.max() + 1) + 1  # for days
    ygrid = np.arange(int(24 / scaling) + 1)  # for hours

    # Ensure the dimensions of 'value' match the expected dimensions for 'xgrid' and 'ygrid'
    if value.shape != (len(ygrid) - 1, len(xgrid) - 1):
        raise ValueError(
            f"Shape of value ({value.shape}) does not match xgrid ({len(xgrid)}) and ygrid ({len(ygrid)}) dimensions."
        )

    ax.pcolormesh(xgrid, ygrid, value, cmap=colormap, vmin=min_val, vmax=max_val)
    ax.set_ylim(int(24 / scaling), 0)
    ax.axis("off")


def plot_heatmap_cf(
    df,
    location,
    carrier,
    scaling,
    colormap,
    min_val,
    max_val,
    year=2013,
    figsize=(14, 5),
    plot_difference=False,
    diff_location=None,
):
    fig, axes = plt.subplots(1, 12, figsize=figsize, sharey=True)
    plt.tight_layout()

    for month, ax in enumerate(axes, start=1):
        # Pass additional parameters if plotting the difference
        day, value = prepare_heatmap_data(
            df,
            month,
            location,
            carrier,
            scaling,
            diff_location=diff_location if plot_difference else None,
        )
        draw_heatmap(ax, day, value, scaling, colormap, min_val, max_val)
        ax.set_title(calendar.month_abbr[month], fontsize=10, pad=3)

    fig.subplots_adjust(
        left=0.05, right=0.98, top=0.9, hspace=0.08, wspace=0.1, bottom=0.15
    )
    cbar_ax = fig.add_axes([0.3, 0.08, 0.4, 0.04])
    norm = mc.Normalize(min_val, max_val)
    cb = fig.colorbar(
        ScalarMappable(norm=norm, cmap=colormap), cax=cbar_ax, orientation="horizontal"
    )
    cb.set_label("Hourly Capacity Factor (%)", size=14)

    fig.text(0.12, 0.12, "Day of the Month", ha="center", va="center", fontsize=14)
    fig.text(
        0.04,
        0.34,
        "Hour of the Day",
        ha="center",
        va="center",
        rotation="vertical",
        fontsize=14,
    )

    annotations = [
        f"Location: {location}"
        if not plot_difference
        else f"Location: {location} vs {diff_location}",
        f"Carrier: {carrier}",
        f"Weather Year: {year}",
        r"Unit: MWh·h$^{-1}$",
    ]
    for i, annotation in enumerate(annotations):
        fig.text(
            0.95,
            0.12 - i * 0.05,
            annotation,
            ha="right",
            va="center",
            fontsize=14,
            color="black",
        )

    fig.savefig("test.pdf", bbox_inches="tight", transparent=True)

In [ ]:
location = "Ireland"
carrier = "solar"
scaling = int(snakemake.config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H
colormap = "RdBu"  # "cividis" # "RdBu"  # https://matplotlib.org/stable/tutorials/colors/colormaps.html
MIN, MAX = -1, 1  #  df["denmark onwind"].min()

plot_heatmap_cf(
    df,
    location,
    carrier,
    scaling,
    colormap,
    MIN,
    MAX,
    diff_location="Denmark",
    plot_difference=True,
)

### Retrieve space-time shifts from model solution

In [ ]:
def retrieve_nb(n, node, rename={}):
    """
    Retrieve nodal energy balance per hour
        -> lines and links are bidirectional AND their subsets are exclusive.
        -> links include fossil gens
    NB {-1} multiplier is a nodal balance sign
    """

    components = ["Generator", "Load", "StorageUnit", "Store", "Link", "Line"]
    nodal_balance = pd.DataFrame(index=n.snapshots)

    for i in components:
        if i == "Generator":
            node_generators = n.generators.query("bus==@node").index
            nodal_balance = nodal_balance.join(n.generators_t.p[node_generators])
        if i == "Load":
            node_loads = n.loads.query("bus==@node").index
            nodal_balance = nodal_balance.join(-1 * n.loads_t.p_set[node_loads])
        if i == "Link":
            node_export_links = n.links.query("bus0==@node").index
            node_import_links = n.links.query("bus1==@node").index
            nodal_balance = nodal_balance.join(-1 * n.links_t.p0[node_export_links])
            nodal_balance = nodal_balance.join(-1 * n.links_t.p1[node_import_links])
            ##################
        if i == "StorageUnit":
            # node_storage_units = n.storage_units.query('bus==@node').index
            # nodal_balance = nodal_balance.join(n.storage_units_t.p_dispatch[node_storage_units])
            # nodal_balance = nodal_balance.join(n.storage_units_t.p_store[node_storage_units])
            continue
        if i == "Line":
            continue
        if i == "Store":
            continue

    nodal_balance = nodal_balance.rename(columns=rename).groupby(level=0, axis=1).sum()

    # Custom groupby function
    def custom_groupby(column_name):
        if column_name.startswith("vcc"):
            return "spatial shift"
        return column_name

    # Apply custom groupby function
    nodal_balance = nodal_balance.groupby(custom_groupby, axis=1).sum()

    # revert nodal balance sign for display
    if "spatial shift" in nodal_balance.columns:
        nodal_balance["spatial shift"] = nodal_balance["spatial shift"] * -1
    if "temporal shift" in nodal_balance.columns:
        nodal_balance["temporal shift"] = nodal_balance["temporal shift"] * -1

    return nodal_balance

In [ ]:
def hourly_curtailment(network, tech, buses):
    """
    Calculate the curtailment for a given technology and bus.

    :param network: The PyPSA network object
    :param tech: Technology type (string)
    :param buses: Buses to consider for the calculation (iterable of strings)
    :param weights: Weights for the calculation (pandas Series or similar)
    :return: Total curtailment for the given technology and buses
    """
    weights = n.snapshot_weightings["generators"]
    gens = network.generators.query("carrier == @tech and bus in @buses").index
    curtailment = (
        (
            network.generators_t.p_max_pu[gens] * network.generators.p_nom_opt[gens]
            - network.generators_t.p[gens]
        )
        .clip(lower=0)
        .multiply(weights, axis=0)
        .sum(axis=1)
    )
    return curtailment

In [ ]:
hourly_curtailment(n, "solar", ["Ireland"])

In [ ]:
retrieve_nb(n, names[1]).columns

In [ ]:
dc1 = names[0]
dc2 = names[1]

In [ ]:
def analyze_datacenter_shifts(n, dc1, dc2):
    """
    Analyze the shifts in energy feed-in and spatial shift for two datacenters.

    :param n: PyPSA Network object
    :param dc1: Name of the first datacenter
    :param dc2: Name of the second datacenter
    :return: Dictionary with analysis results

    NB Positive shift -> sending jobs away; negative shift -> receiving jobs
    """

    # retrieve datacenter data
    def analyze_dc(dc_name):
        feedin = retrieve_nb(n, dc_name)[[f"{dc_name} onwind", f"{dc_name} solar"]]
        curtailment = hourly_curtailment(n, "onwind", [dc_name]) + hourly_curtailment(
            n, "solar", [dc_name]
        )
        spatial_shift = retrieve_nb(n, dc_name)["spatial shift"]

        return {
            "feedin": feedin,
            "curtailment": curtailment,
            "spatial_shift": spatial_shift,
        }

    # Analyze both datacenters
    dc1_analysis = analyze_dc(dc1)
    dc2_analysis = analyze_dc(dc2)

    # Compute differences between wind and solar feed-in
    diff_onwind = (
        dc1_analysis["feedin"][f"{dc1} onwind"]
        - dc2_analysis["feedin"][f"{dc2} onwind"]
    )
    diff_solar = (
        dc1_analysis["feedin"][f"{dc1} solar"] - dc2_analysis["feedin"][f"{dc2} solar"]
    )

    # Collect and store wind and solar hourly potentials
    potentials_dc1 = n.generators_t.p_max_pu[[f"{dc1} onwind", f"{dc1} solar"]]
    potentials_dc2 = n.generators_t.p_max_pu[[f"{dc2} onwind", f"{dc2} solar"]]

    return {
        f"{dc1}": {
            "feedin": dc1_analysis["feedin"],
            "potentials": potentials_dc1,
            "curtailment": dc1_analysis["curtailment"],
            "spatial_shift": dc1_analysis["spatial_shift"],
        },
        f"{dc2}": {
            "feedin": dc2_analysis["feedin"],
            "potentials": potentials_dc2,
            "curtailment": dc2_analysis["curtailment"],
            "spatial_shift": dc2_analysis["spatial_shift"],
        },
        "diff_series": {"onwind": diff_onwind, "solar": diff_solar},
    }

In [ ]:
analyze_datacenter_shifts(n, dc1="Denmark", dc2="Ireland")["diff_series"][
    "solar"
].plot()

In [ ]:
(df[f"Denmark {carrier}"] - df[f"Ireland {carrier}"])[:720].plot()

In [ ]:
# Positive value -> Denmark is better resources; negative values -> Ireland has better resources
value = df[f"Denmark {carrier}"] - df[f"Ireland {carrier}"]
value.plot()

In [ ]:
# Calculate Pearson correlation coefficient
corr_matrix = np.corrcoef(spatial_shift, -value)
corr_matrix

In [ ]:
from scipy import stats

# Assuming value and spatial_shift are your data arrays
correlation_coefficient, p_value = stats.pearsonr(spatial_shift, -value)

print(f"Correlation Coefficient: {correlation_coefficient}")
print(f"p-value: {p_value}")

In [ ]:
corr_matrix = np.corrcoef(df[f"Denmark {carrier}"], df[f"Ireland {carrier}"])
corr_matrix